The final part of week 5 on which i am working with Greek Restaurants in the city of Toronto. The main goal of this project is to recomend good areas that a Greek restaurant could be opened. This project will have 3 core parts. The first one is about exploring the areas of Toronto that have any Greek reataurant the second is to cluster the location of these restaurants and the third to compare these areas in order to recommend the most suitable for a new one.

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from botocore.client import Config
import ibm_boto3
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


Importing the last data set with all the data that used in previous three weeks (scraped from wikipedia and merged with geocoders coordinates )

In [3]:
df=pd.read_csv("C:/Users/Spyros_PC/Desktop/IBM CERT/last.csv")
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Creating a map which includes marks for every Brrough location of Toronto.
Each marker has a popup text which contains all the Neighbourhoods and the Borough at the end.

In [4]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
latitude = 43.6532; longitude = -79.3832
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

Storing the Forsquare Credentials

In [5]:
CLIENT_ID = 'ET0WGFUJUCKRKKNBG4AUPDUMV24FNN3A53UNITL3YPHNV4ZQ' # your Foursquare ID
CLIENT_SECRET = 'GRLHUEH2OG2T5I5MNPYFL0B5RP4ZD3I3T3BFTBDGLOA35KVC' # your Foursquare Secret
ACCESS_TOKEN = 'XBLN52XN45AX2BMUPBWRNN0DTHJCHIJVXCDQDDKEVXMKQVGM' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ET0WGFUJUCKRKKNBG4AUPDUMV24FNN3A53UNITL3YPHNV4ZQ
CLIENT_SECRET:GRLHUEH2OG2T5I5MNPYFL0B5RP4ZD3I3T3BFTBDGLOA35KVC


Let's calculate the average coordinates of the boroughs. We will assume that this is the center of Toronto.

In [6]:
lat_toronto = df['Latitude'].mean()
lon_toronto = df['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

geolocator = Nominatim(user_agent="foursquare_agent")

The geographical coordinates of Toronto are 43.70460773398059, -79.39715291165048


Now the best way to start is by searching for all the Greek buisneses(might be churces and schools as well) that are one and a half kilometer from the center of the available neigbourhoods.

In [7]:
search_query = 'Greek'
radius = 1500
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

Greek .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=ET0WGFUJUCKRKKNBG4AUPDUMV24FNN3A53UNITL3YPHNV4ZQ&client_secret=GRLHUEH2OG2T5I5MNPYFL0B5RP4ZD3I3T3BFTBDGLOA35KVC&ll=43.6532,-79.3832&oauth_token=XBLN52XN45AX2BMUPBWRNN0DTHJCHIJVXCDQDDKEVXMKQVGM&v=20180604&query=Greek&radius=1500&limit=100'

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601d62de0046ef47fed77cb1'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4e8889c961afee1b75b4cc4a',
    'name': 'Jimmy The Greek',
    'location': {'address': '220 Yonge St.',
     'crossStreet': 'in Urban Eatery, Toronto Eaton Centre',
     'lat': 43.6545029224916,
     'lng': -79.38088785568989,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.6545029224916,
       'lng': -79.38088785568989}],
     'distance': 236,
     'postalCode': 'M5B 2H1',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['220 Yonge St. (in Urban Eatery, Toronto Eaton Centre)',
      'Toronto ON M5B 2H1',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d10e941735',
      'name': 'Greek Restaurant',
      'pluralName': 'Greek Restaurants',
      'shortName': 'Greek',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categorie

Get relevant part of JSON and transform it into a pandas dataframe

In [9]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
print(dataframe.shape)
dataframe.head()

(23, 17)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4e8889c961afee1b75b4cc4a,Jimmy The Greek,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",v-1612538590,False,220 Yonge St.,"in Urban Eatery, Toronto Eaton Centre",43.654503,-79.380888,"[{'label': 'display', 'lat': 43.6545029224916,...",236,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St. (in Urban Eatery, Toronto Eaton..."
1,4ba3b374f964a520035638e3,Mary's Greek souvlaki Plate,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",v-1612538590,False,109 McCaul St,McCaul and Dundas,43.653572,-79.390747,"[{'label': 'display', 'lat': 43.653572, 'lng':...",609,M5T 3K5,CA,Toronto,ON,Canada,"[109 McCaul St (McCaul and Dundas), Toronto ON..."
2,4ae7228cf964a520baa821e3,Jimmy the Greek,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",v-1612538590,False,199 Bay St.,in Commerce Court,43.647927,-79.379083,"[{'label': 'display', 'lat': 43.64792662047613...",674,M5L 1G5,CA,Toronto,ON,Canada,"[199 Bay St. (in Commerce Court), Toronto ON M..."
3,5824b2a2569e1b2e982f4451,Jimmy the Greek,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",v-1612538590,False,555 University Ave,NaN,43.657557,-79.388906,"[{'label': 'display', 'lat': 43.657557, 'lng':...",668,NaN,CA,Toronto,ON,Canada,"[555 University Ave, Toronto ON, Canada]"
4,4be82e6a947820a1cc8eb4db,Jimmy The Greek,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",v-1612538590,False,100 Wellington St. W,in Toronto-Dominion Centre,43.647011,-79.382557,"[{'label': 'display', 'lat': 43.647010829377, ...",690,M5J 1J1,CA,Toronto,ON,Canada,[100 Wellington St. W (in Toronto-Dominion Cen...


In [10]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
print(dataframe_filtered.shape)
dataframe_filtered.head()

(23, 15)


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Jimmy The Greek,Greek Restaurant,220 Yonge St.,"in Urban Eatery, Toronto Eaton Centre",43.654503,-79.380888,"[{'label': 'display', 'lat': 43.6545029224916,...",236,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St. (in Urban Eatery, Toronto Eaton...",4e8889c961afee1b75b4cc4a
1,Mary's Greek souvlaki Plate,Greek Restaurant,109 McCaul St,McCaul and Dundas,43.653572,-79.390747,"[{'label': 'display', 'lat': 43.653572, 'lng':...",609,M5T 3K5,CA,Toronto,ON,Canada,"[109 McCaul St (McCaul and Dundas), Toronto ON...",4ba3b374f964a520035638e3
2,Jimmy the Greek,Greek Restaurant,199 Bay St.,in Commerce Court,43.647927,-79.379083,"[{'label': 'display', 'lat': 43.64792662047613...",674,M5L 1G5,CA,Toronto,ON,Canada,"[199 Bay St. (in Commerce Court), Toronto ON M...",4ae7228cf964a520baa821e3
3,Jimmy the Greek,Greek Restaurant,555 University Ave,NaN,43.657557,-79.388906,"[{'label': 'display', 'lat': 43.657557, 'lng':...",668,NaN,CA,Toronto,ON,Canada,"[555 University Ave, Toronto ON, Canada]",5824b2a2569e1b2e982f4451
4,Jimmy The Greek,Greek Restaurant,100 Wellington St. W,in Toronto-Dominion Centre,43.647011,-79.382557,"[{'label': 'display', 'lat': 43.647010829377, ...",690,M5J 1J1,CA,Toronto,ON,Canada,[100 Wellington St. W (in Toronto-Dominion Cen...,4be82e6a947820a1cc8eb4db


Now lets see how many are restaurants from the returned Foursqare venues.

In [11]:
dataframe_filtered['categories'].value_counts()

Greek Restaurant       16
Church                  5
Government Building     1
Hot Dog Joint           1
Name: categories, dtype: int64

Now we know there are greek restaurants in Toronto, so the next step is to locate the Neighborhoods with their coordinates.
We need this in order to cluster these neighborhoods to make the recomendations.

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

At first we create a dataframe containing all venues

In [13]:

toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                latitudes=df['Latitude'],
                                longitudes=df['Longitude'])
toronto_venues

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
...,...,...,...,...,...,...,...
2109,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Jim & Maria's No Frills,43.631152,-79.518617,Grocery Store
2110,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Royal Canadian Legion #210,43.628855,-79.518903,Social Club
2111,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
2112,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym


We perfom one hot encoding to our data in order to use them for the clustering algorithm

In [14]:
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot

(2114, 274)


,Neighborhoods,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2109,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2110,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2111,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2112,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The next step is to calculate the mean column values per neighborhood because more than one restaurant can exist in the same neughborhood.

In [15]:
to_onehot = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

#print(to_onehot.shape)
to_onehot

,Neighborhoods,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.043478,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.000000,0.0
92,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
93,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
94,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0



getting the columns that will be used by the clustering algoritm

In [16]:
greek_df = to_onehot[['Neighborhoods','Greek Restaurant']][to_onehot['Greek Restaurant']>0]
greek_df.head()

,Neighborhoods,Greek Restaurant
4,"Bedford Park, Lawrence Manor East",0.043478
5,Berczy Park,0.018182
19,Davisville,0.027778
28,"Fairview, Henry Farm, Oriole",0.015152
29,"First Canadian Place, Underground city",0.010000


The clustering algorith is K-Means and implemented up next

In [17]:
from sklearn.cluster import KMeans
num_clusters = 3

clustering = greek_df.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=1)
kmeans.fit_transform(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([2, 0, 2, 0, 0, 0, 1, 0])

Adding the Cluster Numbers to the data

In [18]:
greek_df.rename(columns = {'Neighborhoods': 'Neighbourhood'}, inplace = True)
greek_df['Cluster']=kmeans.labels_
greek_df

,Neighbourhood,Greek Restaurant,Cluster
4,"Bedford Park, Lawrence Manor East",0.043478,2
5,Berczy Park,0.018182,0
19,Davisville,0.027778,2
28,"Fairview, Henry Farm, Oriole",0.015152,0
29,"First Canadian Place, Underground city",0.010000,0
48,"Little Portugal, Trinity",0.023256,0
80,"The Danforth West, Riverdale",0.166667,1
83,"Toronto Dominion Centre, Design Exchange",0.010000,0


And finaly we will merge them to the main dataframe in order to match every row with its coordinates.
This will allow the visualization of the clusters on a map.

In [20]:
greek_df=pd.merge(greek_df, toronto_venues, on="Neighbourhood")
greek_df=greek_df[greek_df['Venue Category']=='Greek Restaurant']
greek_df

,Neighbourhood,Greek Restaurant,Cluster,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
9,"Bedford Park, Lawrence Manor East",0.043478,2,43.733283,-79.419750,Karbouzi Greek Taverna,43.736204,-79.420359,Greek Restaurant
43,Berczy Park,0.018182,0,43.644771,-79.373306,Alexandro's World Famous Gyros,43.641663,-79.375214,Greek Restaurant
98,Davisville,0.027778,2,43.704324,-79.388790,souvlaki express,43.707378,-79.389848,Greek Restaurant
168,"Fairview, Henry Farm, Oriole",0.015152,0,43.778517,-79.346556,Jimmy The Greek,43.778245,-79.343322,Greek Restaurant
222,"First Canadian Place, Underground city",0.010000,0,43.648429,-79.382280,Estiatorio Volos,43.650329,-79.384533,Greek Restaurant
290,"Little Portugal, Trinity",0.023256,0,43.647927,-79.419750,Mamakas Taverna,43.645908,-79.419654,Greek Restaurant
324,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
326,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
332,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Messini Authentic Gyros,43.677704,-79.350480,Greek Restaurant
338,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Christina's On The Danforth,43.678240,-79.349185,Greek Restaurant


we visualize the 3 clusters on the map.

In [21]:
map_clusters = folium.Map(location=[lat_toronto, lon_toronto],zoom_start=11)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster, neig in zip(greek_df['Neighbourhood Latitude'], greek_df['Neighbourhood Longitude'], greek_df['Cluster'],greek_df['Neighbourhood']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
        popup='Neighborhood(s): {} \nCluster_Number: {}'.format(neig,str(cluster)),
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [22]:
greek_df.sort_values(by='Cluster').reset_index(drop=True)


,Neighbourhood,Greek Restaurant,Cluster,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.018182,0,43.644771,-79.373306,Alexandro's World Famous Gyros,43.641663,-79.375214,Greek Restaurant
1,"Fairview, Henry Farm, Oriole",0.015152,0,43.778517,-79.346556,Jimmy The Greek,43.778245,-79.343322,Greek Restaurant
2,"First Canadian Place, Underground city",0.010000,0,43.648429,-79.382280,Estiatorio Volos,43.650329,-79.384533,Greek Restaurant
3,"Little Portugal, Trinity",0.023256,0,43.647927,-79.419750,Mamakas Taverna,43.645908,-79.419654,Greek Restaurant
4,"Toronto Dominion Centre, Design Exchange",0.010000,0,43.647177,-79.381576,Estiatorio Volos,43.650329,-79.384533,Greek Restaurant
5,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
6,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
7,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Messini Authentic Gyros,43.677704,-79.350480,Greek Restaurant
8,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Christina's On The Danforth,43.678240,-79.349185,Greek Restaurant
9,"The Danforth West, Riverdale",0.166667,1,43.679557,-79.352188,Astoria Shish Kebob House,43.677596,-79.351738,Greek Restaurant


In [23]:
greek_df.groupby(['Cluster','Neighbourhood'])['Greek Restaurant'].count().to_frame()


Greek Restaurant
Cluster Neighbourhood                                             
0       Berczy Park                                              1
        Fairview, Henry Farm, Oriole                             1
        First Canadian Place, Underground city                   1
        Little Portugal, Trinity                                 1
        Toronto Dominion Centre, Design Exchange                 1
1       The Danforth West, Riverdale                             7
2       Bedford Park, Lawrence Manor East                        1
        Davisville                                               1

Conclusion

The point of this project was to locate the areas with Greek restaurants. By this way we can locate where there are pieces of Greek culture in the city of Toronto and there it could be a good opportunity for someone o start his Greek restaurant. Even more clustering helped to recommend neighborhoods on which the competitions is the lowest possible.

Its seems that the most (7) greek restaurants are placed in cluster 1 .
Up next there other biggest cluster is 0 with 5 reastaurants on Berczy Park, First Canadian Place, Underground city,Little Portugal, Trinity Toronto Dominion Centre and Design Exchange neighborhoods.
Finaly the least restaurants 1 are placed in cluster 2 on Bedford Park, Lawrence Manor East and Davisville.
So the most suitable place to open a greek restaurant is on Bedford Park, Lawrence Manor East and Davisville neighborhoods.